In [2]:
import tensorflow as tf
import numpy as np

In [3]:
t = 5
def load_data():
    for i in range(t):
        with open("GFSC/reuters_views/view" + str(i) + ".npy", 'rb') as f:
            tmp = tf.cast([np.load(f)], dtype=tf.float32)
            if i == 0:
                x = tf.linalg.matmul(tmp, tmp, transpose_b=True)
            else:
                x = tf.concat([x, tf.linalg.matmul(tmp, tmp, transpose_b=True)], axis=0)
    return x
X = load_data()
print(X.shape)

(5, 1200, 1200)


In [18]:
k = 6
steps = 200
alphas, betas, gamma = [1.0], [1000.0], 0.01
n = X.shape[2]

In [19]:
def fusion():
    f = None
    for alpha in alphas:
        for beta in betas:
            reg = gamma / (4.0 * beta)
            wv, wv_u = tf.cast([1.0 / t for _ in range(t)], dtype=tf.float32), None
            i_alpha = tf.math.multiply(tf.eye(n, dtype=tf.float32), alpha)
            i_beta = tf.math.multiply(tf.eye(n, dtype=tf.float32), beta)
            s = tf.eye(n, dtype=tf.float32)
            for step in range(steps):
                s = tf.where(s > 0, s, tf.zeros(s.shape))
                s = tf.math.divide(tf.math.add(s, tf.transpose(s)), 2.0)
                z_sum = tf.zeros(s.shape)
                for i in range(t):
                    z = tf.linalg.solve(
                        tf.math.add(tf.math.add(X[i], i_alpha), tf.math.multiply(i_beta, wv[i])),
                        tf.math.add(tf.math.multiply(s, tf.math.multiply(wv[i], beta)), X[i]))
                    z = tf.where(z > 0, z, tf.zeros(z.shape))
                    z = tf.math.divide(tf.math.add(z, tf.transpose(z)), 2.0)
                    w = tf.math.divide(0.5, tf.norm(tf.math.subtract(z, s)))
                    wv_u = tf.reshape(w, [1]) if i == 0 else tf.concat([wv_u, [w]], axis=0)
                    z = tf.math.multiply(z, w)
                    z_sum = tf.math.add(z_sum, z)

                l = tf.math.subtract(tf.linalg.diag(tf.math.reduce_sum(s, 0)), s)
                (_, f) = tf.linalg.eigh(l)
                f = tf.slice(f, [0, 0], [f.shape[0], k])
                p = tf.norm(tf.math.subtract(f[0][:], f), axis=1, keepdims=True)
                for i in range(1, n):
                     p = tf.concat([p, tf.norm(tf.math.subtract(f[0][:], f), axis=1, keepdims=True)], 1)
                p = tf.math.multiply(tf.math.square(p), reg)
                wv = wv_u
                s_old = s
                s = tf.math.divide((tf.math.subtract(z_sum, p)), tf.math.reduce_sum(wv, 0))
                if step > 5 and tf.math.less(tf.math.divide(tf.norm(tf.math.subtract(s, s_old)), tf.norm(s_old)), 1.e-3):
                    break
    return f

In [20]:
F = fusion()
print(F)

tf.Tensor(
[[ 0.02887338 -0.0068409   0.00262845 -0.02016393 -0.02948655 -0.0260959 ]
 [ 0.02887817  0.22645688 -0.09875882 -0.01463154  0.01879995 -0.01402079]
 [ 0.02887604  0.03631481  0.1006254  -0.07639755 -0.00193545  0.03355392]
 ...
 [ 0.02886708 -0.01025294 -0.01158178  0.00119391  0.00980991 -0.05308314]
 [ 0.02886656 -0.00464873 -0.00155176 -0.0056548  -0.00696376  0.01128093]
 [ 0.02886647 -0.00687443 -0.00237176 -0.00442624 -0.0065471   0.00279775]], shape=(1200, 6), dtype=float32)
